In [5]:
import pygame
import random
import math

In [6]:
pygame.init()

FPS = 60
width, height = 600, 600
rows, cols = 4, 4

rect_height = height // rows
rect_width = width // cols


outline_colour = (187, 173, 160)
outline_thickness = 10
background_colour = (205, 192, 180)
font_colour = (119, 110, 101)


In [7]:
class Window:
    def __init__(self):
        self.window = pygame.display.set_mode((width, height))
        pygame.display.set_caption('2048')
        self.font = pygame.font.SysFont('comicsans', 60, bold=True)
        self.move_vel = 20

    def mainloop(self):
        self.clock = pygame.time.Clock()
        run = True
        
        self.tiles = self.generate_tiles()

        while run:
            self.clock.tick(FPS)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                    break
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_LEFT:
                        self.move_tiles(self.tiles, "left") 
            self.draw(self.tiles)
        
        pygame.quit()

    
    def draw(self, tiles):
        self.window.fill(background_colour)

        for tile in tiles.values():
            tile.draw(self.window)
        self.draw_grid()
        pygame.display.update()
        

    def draw_grid(self):
        for row in range(1, rows):
            y = row*rect_height
            pygame.draw.line(self.window, outline_colour, (0,y), (width,y), outline_thickness)
        for col in range(1,cols):
            x = col*rect_width
            pygame.draw.line(self.window, outline_colour, (x,0), (x, height), outline_thickness)
        pygame.draw.rect(self.window, outline_colour, (0, 0, width, height), outline_thickness)
    
    def generate_tiles(self):
        tiles = {}
        for _ in range(2):
            row, col = self.get_random_pos(tiles)
            tiles[f"{row}, {col}"] = Tile(2, row, col)

        return tiles
    
    def move_tiles(self, tiles, direction):
        updated = True
        blocks = set()
        
        if direction == "left":
            sort_func = lambda tile:tile.col
            reverse = False
            delta = (-self.move_vel, 0)
            boundary_check = lambda tile: tile.col == 0
            get_next_tile = lambda tile: self.tiles.get(f'{tile.row}, {tile.col-1}')
            merge_check = lambda tile, next_tile: tile.x > next_tile.x + self.move_vel
            move_check = lambda tile, next_tile: tile.x > next_tile.x + rect_width + self.move_vel
            ceil = True
        elif direction == "right":
            pass
        elif direction == "up":
            pass
        elif direction == "down":
            pass

        while updated:
            self.clock.tick(FPS)
            updated = False
            sorted_tiles = sorted(tiles.values(), key=sort_func, reverse=reverse)

            for i,tile in enumerate(sorted_tiles):
                if boundary_check(tile):
                    continue
                next_tile = get_next_tile(tile)
                if not(next_tile):
                    tile.move(delta)
                elif (tile.value == next_tile.value
                     and tile not in blocks
                     and next_tile not in blocks):
                    
                    if merge_check(tile, next_tile):
                        tile.move(delta)
                    else:
                        next_tile.value *= 2
                        sorted_tiles.pop(i)
                        blocks.add(next_tile)

                elif move_check(tile, next_tile):
                    tile.move(delta)
                else: continue 
                tile.set_pos(ceil)
                updated = True
            self.update_tiles(self.window, tiles, sorted_tiles)
        self.end_move(tiles)
    
    def end_move(self, tiles):
        if len(tiles) == rows * cols:
            return "lost"
        row, col = self.get_random_pos(tiles)
        tiles[f'{row}, {col}'] = Tile(random.choice([2,4]), row, col)
        return "continue"
    
    def update_tiles(self, window, tiles, sorted_tiles):
        tiles.clear()
        for tile in sorted_tiles:
            tiles[f'{tile.row}, {tile.col}'] = tile

        self.draw(tiles)


    
    def get_random_pos(self, tiles):
        row, col = None, None
        while True:
            row = random.randrange(0, rows)
            col = random.randrange(0, cols)
            
            if f"{row}, {col}" not in tiles:
                break
        return row, col
class Tile:
    colours = [
        (237, 229, 218),
        (238, 225, 201),
        (243, 178, 122),
        (246, 150, 101),
        (247, 124, 95),
        (247, 95, 59),
        (237, 208, 115),
        (237, 204, 99),
        (236, 202, 80)
    ]
        
    def __init__(self, value, row, col):
        self.font = pygame.font.SysFont('comicsans', 60, bold=True)
        self.value = value
        self.row = row
        self.col = col  
        self.x = col*rect_width
        self.y = row*rect_height

    def get_colour(self):
        colour_index = int(math.log2(self.value)) - 1
        colour = self.colours[colour_index]
        return colour

    def draw(self, window):
        colour = self.get_colour()
        pygame.draw.rect(window, colour, (self.x, self.y, rect_width, rect_height))
        text = self.font.render(str(self.value), 1, font_colour)
        window.blit(text, 
                    (self.x + (rect_width - text.get_width())//2,
                    self.y + (rect_height - text.get_height())//2))
    
    def move(self, delta):
        self.x += delta[0]
        self.y += delta[1]

    def set_pos(self, ceil=False):
        if ceil:
            self.row = math.ceil(self.y/rect_height)
            self.col = math.ceil(self.x/rect_width)
        else:
            self.row = math.floor(self.y/rect_height)
            self.col = math.floor(self.x/rect_width)



In [8]:
app = Window()
if __name__ == "__main__":
    app.mainloop()